In [1]:
# Requires brainvisa

In [2]:
import anatomist.api as anatomist
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from soma import aims
import json
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np
import glob

In [4]:
a = anatomist.Anatomist()

create qapp
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module meshsplit
loading module palettecontrols
loading module volumepalettes


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


In [5]:
Rspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

Lspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_left/meshes/Lspam_model_meshes_1.arg"


json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_gridsearch.json"

path_summary = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/summary"


In [6]:
with open(json_regions) as f:
    regions = json.load(f)

In [7]:
files_to_display = glob.glob(f"{path_summary}/*")

In [8]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [9]:
files_to_display

['/neurospin/dico/data/deep_folding/current/models/Champollion_V0/summary/isRightHanded60vsLeftHanded60_hcp_without_residualization.csv',
 '/neurospin/dico/data/deep_folding/current/models/Champollion_V0/summary/isStronglyRightHanded_hcp_without_residualization.csv',
 '/neurospin/dico/data/deep_folding/current/models/Champollion_V0/summary/Sex_ukb_without_residualization.csv',
 '/neurospin/dico/data/deep_folding/current/models/Champollion_V0/summary/SexSmallBrain_ukb_without_residualization.csv',
 '/neurospin/dico/data/deep_folding/current/models/Champollion_V0/summary/isBigBrain_ukb_sex_balanced.csv',
 '/neurospin/dico/data/deep_folding/current/models/Champollion_V0/summary/isStronglyRightHandedvsOthers_hcp_without_residualization.csv',
 '/neurospin/dico/data/deep_folding/current/models/Champollion_V0/summary/isBigBrain_ukb_without_residualization.csv',
 '/neurospin/dico/data/deep_folding/current/models/Champollion_V0/summary/isRightHanded_hcp_without_residualization.csv',
 '/neurospi

In [10]:
file_to_display = files_to_display[-2]
file_to_display

'/neurospin/dico/data/deep_folding/current/models/Champollion_V0/summary/isOld_ukb_without_residualization.csv'

In [11]:
df = pd.read_csv(file_to_display, index_col=0)[["region", "roc_auc", "roc_auc_std"]]
res = df.groupby(['region']).mean()
res["side"] = res.index.str.split('_').str[-1]
res = res.reset_index()

In [12]:
res = res[~res.region.str.contains("fronto-parietal")]
res = res[~res.region.str.contains("CINGULATE")]

In [13]:
"CINGULATE_left".replace("CINGULATE", "CINGULATE.")

'CINGULATE._left'

In [14]:
def get_sulci(region):
    region = region.replace("CINGULATE", "CINGULATE.")
    region = region.replace("ORBITAL", "S.Or.")
    list_sulci = list(regions['brain'][f"{region}"].keys())
    list_sulci = [x.replace("paracingular.", "S.F.int.") for x in list_sulci]
    return list_sulci

In [15]:
get_sulci("CINGULATE_left")

['S.F.int._left', 'F.C.M.ant._left']

In [16]:
res.head()

,region,roc_auc,roc_auc_std,side
2,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,0.831279,0.014145,left
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,0.850777,0.007148,right
4,F.C.M.post.-S.p.C._left,0.667601,0.023119,left
5,F.C.M.post.-S.p.C._right,0.680090,0.015131,right
6,F.Coll.-S.Rh._left,0.816352,0.020166,left


In [17]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,region,roc_auc,roc_auc_std,side,sulcus
2,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,0.831279,0.014145,left,"[F.C.L.p._left, F.C.L.r.sc.ant._left, F.C.L.r...."
3,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,0.850777,0.007148,right,"[F.C.L.p._right, F.C.L.r.sc.ant._right, F.C.L...."
4,F.C.M.post.-S.p.C._left,0.667601,0.023119,left,"[F.C.M.post._left, S.C.LPC._left, S.p.C._left]"
5,F.C.M.post.-S.p.C._right,0.680090,0.015131,right,"[F.C.M.post._right, S.C.LPC._right, S.p.C._right]"
6,F.Coll.-S.Rh._left,0.816352,0.020166,left,"[F.Coll._left, S.Rh._left]"


In [18]:
res = res.sort_values(by="roc_auc", ascending=False)
res = res.explode("sulcus")
res[res.region.str.contains("S.T.s.")]

,region,roc_auc,roc_auc_std,side,sulcus
45,S.T.i.-S.T.s.-S.T.pol._right,0.767118,0.011303,right,S.T.i.ant._right
45,S.T.i.-S.T.s.-S.T.pol._right,0.767118,0.011303,right,S.T.i.post._right
45,S.T.i.-S.T.s.-S.T.pol._right,0.767118,0.011303,right,S.T.s._right
45,S.T.i.-S.T.s.-S.T.pol._right,0.767118,0.011303,right,S.T.pol._right
47,S.T.s._right,0.746398,0.010286,right,S.T.s._right
44,S.T.i.-S.T.s.-S.T.pol._left,0.733114,0.005381,left,S.T.i.ant._left
44,S.T.i.-S.T.s.-S.T.pol._left,0.733114,0.005381,left,S.T.i.post._left
44,S.T.i.-S.T.s.-S.T.pol._left,0.733114,0.005381,left,S.T.s._left
44,S.T.i.-S.T.s.-S.T.pol._left,0.733114,0.005381,left,S.T.pol._left
46,S.T.s._left,0.724051,0.009999,left,S.T.s._left


In [19]:
for _, row in res.iterrows():
    print(row.sulcus)

F.Cal.ant.-Sc.Cal._left
S.O.p._left
S.Li.ant._left
S.Li.post._left
F.P.O._left
S.Pa.t._left
S.Cu._left
F.Cal.ant.-Sc.Cal._left
S.O.p._left
F.C.L.p._right
F.C.L.r.sc.ant._right
F.C.L.r.sc.post._right
INSULA_right
F.C.L.a._right
F.C.L.p._left
F.C.L.r.sc.ant._left
F.C.L.r.sc.post._left
INSULA_left
F.C.L.a._left
F.Coll._left
S.Rh._left
F.Cal.ant.-Sc.Cal._right
S.O.p._right
S.Li.ant._right
S.Li.post._right
F.Coll._right
S.Rh._right
S.T.i.ant._right
S.T.i.post._right
S.T.s._right
S.T.pol._right
S.Or._right
S.Olf._right
S.F.int._left
S.R.inf._left
S.R.sup._left
S.Or._left
S.Olf._left
F.C.M.ant._left
S.F.int._left
F.P.O._right
S.Pa.t._right
S.Cu._right
F.Cal.ant.-Sc.Cal._right
S.O.p._right
S.T.s._right
S.s.P._left
S.Pa.int._left
S.s.P._right
S.Pa.int._right
S.T.i.ant._left
S.T.i.post._left
S.T.s._left
S.T.pol._left
F.C.M.ant._right
S.F.int._right
S.T.s._left
S.T.i.ant._left
S.T.i.post._left
S.O.T.lat.ant._left
S.O.T.lat.int._left
S.O.T.lat.med._left
S.O.T.lat.post._left
S.T.i.ant._right
S.T.i.

In [20]:
res[res.sulcus=="S.F.orbitaire._right"]

,region,roc_auc,roc_auc_std,side,sulcus
33,S.F.marginal-S.F.inf.ant._right,0.647758,0.016589,right,S.F.orbitaire._right


In [21]:
res

,region,roc_auc,roc_auc_std,side,sulcus
52,Sc.Cal.-S.Li._left,0.856327,0.021827,left,F.Cal.ant.-Sc.Cal._left
52,Sc.Cal.-S.Li._left,0.856327,0.021827,left,S.O.p._left
52,Sc.Cal.-S.Li._left,0.856327,0.021827,left,S.Li.ant._left
52,Sc.Cal.-S.Li._left,0.856327,0.021827,left,S.Li.post._left
10,F.P.O.-S.Cu.-Sc.Cal._left,0.855534,0.012620,left,F.P.O._left
...,...,...,...,...,...
38,S.Pe.C._left,0.583277,0.019776,left,S.Pe.C.inter._left
38,S.Pe.C._left,0.583277,0.019776,left,S.Pe.C.marginal._left
38,S.Pe.C._left,0.583277,0.019776,left,S.Pe.C.sup._left
38,S.Pe.C._left,0.583277,0.019776,left,S.Pe.C.median._left


In [22]:
def set_color_property(res, side):
    global dic

    if side == "L":
        spam_model_file = Lspam_model
    else:
        spam_model_file = Rspam_model
        
    dic[f"aims{side}"] = aims.read(spam_model_file)

    for vertex in dic[f"aims{side}"].vertices():
        vertex['roc_auc'] = 0.

    for _, row in res.iterrows():
        for vertex in dic[f"aims{side}"].vertices():
            vname = vertex.get('name')
            if vname == row.sulcus:
                    if row.roc_auc-row.roc_auc_std < 0.5:
                        vertex['roc_auc'] = 0.
                    elif vertex['roc_auc'] != 0.: 
                        vertex['roc_auc'] = min(vertex['roc_auc'],
                                                row.roc_auc)
                    else:
                        vertex['roc_auc'] = row.roc_auc
    
    dic[f"ana{side}"] = a.toAObject(dic[f"aims{side}"])

    dic[f"ana{side}"].setColorMode(dic[f"ana{side}"].PropertyMap)
    dic[f"ana{side}"].setColorProperty('roc_auc')
    dic[f"ana{side}"].notifyObservers()
    
                
def visualize_whole_hemisphere(view_quaternion, side, i):
    global block
    global dic
    try:
        block
    except NameError:
        block = a.createWindowsBlock(4)

    dic[f"win{i}"] = a.createWindow('3D',
                                    block=block,
                                    no_decoration=True,
                                    options={'hidden': 1})
    dic[f"win{i}"].addObjects(dic[f"ana{side}"])
    dic[f"ana{side}"].setPalette("green_yellow_red",
                              minVal=0.5, absoluteMode=True)
    
    dic[f"win{i}"].camera(view_quaternion=view_quaternion)

middle_view = [0.5, -0.5, -0.5, 0.5]
side_view = [0.5, 0.5, 0.5, 0.5]
bottom_view = [0, -1, 0, 0]
top_view = [0, 0, 0, -1]
 
def visualize_whole(res, side, start):
    set_color_property(res, side)
    visualize_whole_hemisphere(middle_view if side == "L" else side_view, side, start+0)
    visualize_whole_hemisphere(top_view, side, start+1)
    visualize_whole_hemisphere(bottom_view, side, start+2)
    visualize_whole_hemisphere(side_view if side == "L" else middle_view, side, start+3)


In [23]:
dic = {}

visualize_whole(res, "L", 0)
visualize_whole(res, "R", 4)

Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60
Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60


Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.


QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout
